In [44]:
# importing the dependencies
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from numpy import asarray
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from tensorflow.math import confusion_matrix
from tensorflow.keras.models import load_model
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
# Data Preprocessing
DATA_PATH = os.path.join('dataset')

digits = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

no_labels = 10

In [13]:
# data and label mapping
X, y = [], []
for label in range(no_labels):
    path = os.path.join(DATA_PATH, str(label))
    for file in os.listdir(path):
        image = Image.open(path + '/' + file)
        data = asarray(image)
        X.append(data)
        y.append(label)

In [18]:
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(1075, 128, 128) (1075,)


In [23]:
# train and test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [24]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1021, 128, 128) (54, 128, 128) (1021,) (54,)


In [30]:
# scaling the values (normalization)
X_train = X_train / 255
X_test = X_test / 255

In [35]:
# Building Neural Network
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128)),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid'),
])

In [36]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy']
)

In [37]:
#training the neural network
model.fit(X_train, y_train, epochs=40)

Epoch 1/50
32/32 [==============================] - 4s 11ms/step - loss: 2.3508 - accuracy: 0.2586
Epoch 2/50
32/32 [==============================] - 0s 11ms/step - loss: 1.3564 - accuracy: 0.5240
Epoch 3/50
32/32 [==============================] - 0s 11ms/step - loss: 1.0765 - accuracy: 0.6229
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.8758 - accuracy: 0.7111
Epoch 5/50
32/32 [==============================] - 0s 11ms/step - loss: 0.7435 - accuracy: 0.7561
Epoch 6/50
32/32 [==============================] - 0s 12ms/step - loss: 0.6780 - accuracy: 0.7806
Epoch 7/50
32/32 [==============================] - 0s 14ms/step - loss: 0.6089 - accuracy: 0.8129
Epoch 8/50
32/32 [==============================] - 0s 11ms/step - loss: 0.5250 - accuracy: 0.8355
Epoch 9/50
32/32 [==============================] - 0s 11ms/step - loss: 0.5083 - accuracy: 0.8306
Epoch 10/50
32/32 [==============================] - 0s 15ms/step - loss: 0.4093 - accuracy: 0.8825
Epoch 11/

In [38]:
model.save('dense_NN.h5')

In [39]:
loss, accuracy = model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.8270 - accuracy: 0.8333


In [40]:
# Making prediction on X_test
y_pred = model.predict(X_test)

2/2 [==============================] - 0s 4ms/step


In [41]:
# Converting the prediction probabilities to class label for all test cases
y_pred_labels = [np.argmax(i) for i in  y_pred]

In [42]:
y_pred_labels

[6,
 9,
 8,
 5,
 2,
 8,
 2,
 2,
 5,
 9,
 0,
 1,
 7,
 1,
 9,
 6,
 4,
 1,
 6,
 6,
 5,
 2,
 1,
 5,
 6,
 4,
 6,
 6,
 5,
 4,
 8,
 9,
 8,
 1,
 3,
 3,
 4,
 9,
 5,
 6,
 6,
 1,
 3,
 4,
 8,
 8,
 7,
 3,
 1,
 4,
 4,
 0,
 2,
 0]

In [43]:
# Building Confusion Matrix
conf_mat = confusion_matrix(y_test, y_pred_labels)
print(conf_mat)

tf.Tensor(
[[2 0 0 0 0 0 0 0 0 0]
 [0 7 0 0 0 0 0 0 0 0]
 [0 0 5 2 0 0 0 0 0 0]
 [0 0 0 2 2 0 0 0 1 0]
 [1 0 0 0 5 0 0 0 1 0]
 [0 0 0 0 0 6 0 0 1 0]
 [0 0 0 0 0 0 9 0 0 0]
 [0 0 0 0 0 0 0 2 0 1]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 4]], shape=(10, 10), dtype=int32)


In [45]:
accuracy_score(y_test, y_pred_labels)

0.8333333333333334